In [8]:
from langchain.llms import GooglePalm

api_key = 'Your API Key Here'

llm = GooglePalm(google_api_key=api_key, temperature=0.7)

In [9]:
Song = llm("Write me a song about John")
print(Song)

(Verse 1)
John, John, the man of the hour
Always there to lend a hand
He's got a heart of gold
And a smile that never fades

(Chorus)
John, John, you're one of a kind
A true friend to all
We're so lucky to have you
In our lives

(Verse 2)
John, John, you always know how to make us laugh
With your funny stories and jokes
You're always there for us
When we need you the most

(Chorus)
John, John, you're one of a kind
A true friend to all
We're so lucky to have you
In our lives

(Bridge)
We'll cherish the memories we've made
With you, John
You'll always hold a special place in our hearts

(Chorus)
John, John, you're one of a kind
A true friend to all
We're so lucky to have you
In our lives


In [13]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/Users/rafaelzieganpalg/Projects/LangChain_Project/Rafael_QA/Stan_Data_UPG.csv',source_column='question')

data = loader.load()

In [14]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceInstructEmbeddings()
Vector_db=FAISS.from_documents(documents=data, embedding=embeddings)

load INSTRUCTOR_Transformer


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
max_seq_length  512


In [15]:
retriever = Vector_db.as_retriever()
r_docs=retriever.get_relevant_documents("Who is Beyonce?")
r_docs

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content=': 4\nquestion: In which decade did Beyonce become famous?\ntext: late 1990s', metadata={'source': 'In which decade did Beyonce become famous?', 'row': 4}),
 Document(page_content=': 8\nquestion: When did Beyoncé rise to fame?\ntext: late 1990s', metadata={'source': 'When did Beyoncé rise to fame?', 'row': 8}),
 Document(page_content=": 417\nquestion: Why is Beyoncé known as the centerpiece of Destiny's Child?\ntext: Her vocal abilities", metadata={'source': "Why is Beyoncé known as the centerpiece of Destiny's Child?", 'row': 417}),
 Document(page_content=': 546\nquestion: Beyonce has a fan base that is referred to as what?\ntext: The Bey Hive', metadata={'source': 'Beyonce has a fan base that is referred to as what?', 'row': 546})]

In [16]:
from langchain.prompts import PromptTemplate
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [17]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        input_key="query",
        return_source_documents= True,
        chain_type_kwargs= chain_type_kwargs
)

In [18]:
chain("Who is Beyonce")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'Who is Beyonce',
 'result': 'married to? text: Jay-Z',
 'source_documents': [Document(page_content=': 4\nquestion: In which decade did Beyonce become famous?\ntext: late 1990s', metadata={'source': 'In which decade did Beyonce become famous?', 'row': 4}),
  Document(page_content=": 417\nquestion: Why is Beyoncé known as the centerpiece of Destiny's Child?\ntext: Her vocal abilities", metadata={'source': "Why is Beyoncé known as the centerpiece of Destiny's Child?", 'row': 417}),
  Document(page_content=': 546\nquestion: Beyonce has a fan base that is referred to as what?\ntext: The Bey Hive', metadata={'source': 'Beyonce has a fan base that is referred to as what?', 'row': 546}),
  Document(page_content=': 8\nquestion: When did Beyoncé rise to fame?\ntext: late 1990s', metadata={'source': 'When did Beyoncé rise to fame?', 'row': 8})]}